# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a **classification** problem since the output is binary (i.e. discrete): students who need early intervention vs. students who do not. Regression problems have continuous output (function values for example).

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [235]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print student_data.head()

Student data read successfully!
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        yes       no       5         3      3    1    1      3        4   
2  ...        yes       no       4         3      2    2    3      3       10   
3  ...        yes      yes       3         2      2    1    1      5        2   
4  ...         no       no       4         3      2    1    2      5        4   

  passed  
0     no  
1 

### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [236]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
# Attention: Number of features is number of columns minus one because of the target column "passed"
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed']=="yes"].shape[0]

# TODO: Calculate failing students
n_failed = n_students - n_passed
# alternatively:
# n_failed = student_data[student_data['passed']=="no"].size

# TODO: Calculate graduation rate
grad_rate = 100.0*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [237]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

#d = {'yes': 1, 'no': 0}
#y_all2 = y_all.map(d)

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [238]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [239]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all,y_all, train_size = num_train, random_state=15)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
I will look at the algorithms: Decision Tree, Support Vector Machine and k-Nearest Neighbor
* **Decision Trees**
   - **Real world application**
      - Decision trees can for example be used in credit risk analysis where a number of data points on an applicant are used to decide whether to grant a loan or not (see e.g. http://www.ibm.com/support/knowledgecenter/SSLVMB_22.0.0/com.ibm.spss.statistics.cs/spss/tutorials/trees_credit_intro1.htm, http://dni-institute.in/blogs/decision-tree-steps-to-build-predictive-model/). Data points to be used include: loan amount, loan duration, applicant's salary, age, loan history/number of previous loans, other outstanding debt, marital status, profession or education level etc.
      
   - **Strengths:**
      - Decision trees can work if the data is a mix of categorical and continuous features (only one dimension is looked at at a given step. Decision tree do not use geometric ideas like for example SVMs (where hyperplanes in features space are used to separate data)
      - Decision trees are easy to interpret
      - Decision trees are fast to train and to evaluate on testing data
      - Decision trees work on binary and multi-output data
      - There is no pre-processing (e.g. data scaling) required (since dimensions are not "mixed")
   - **Weaknesses:**
      - Decision trees easily show signs of overfitting (if not told otherwise they will become very granular on the training data, which may result in low accuracy on testing data)
      - Decision trees are also sometimes sensitive on the type or content of input data (the algorithm is relatively brute force)
      - In some problems (e.g. with purely continuous input data) the simple cartesian product logic (dealing with one dimension at a time) could result in too complex trees where an SVM or linear model could work much better
   - **Why is this a good model?:**
      - Decision trees could work well here because of the mixture of continuous and categorical input data
      - Also decision trees could be accurate enough for the admittedly important question at hand (as long as overfitting is prevented)
   
   
* **Support Vector Machines**
   - **Real world application**:
      - SVMs are used in biosciences, for example for classification/structure prediction of proteins (https://noble.gs.washington.edu/papers/noble_support.pdf) or genes through image analysis. SVMs work here because of high dimensionality of the data (images) as well as noise present in the images (SVMs are quite robust to noise). 
      - Other applications include: hand-written text classification (e.g. http://file.scirp.org/pdf/ENG_2013072909321491.pdf)

   - **Strengths:**
      - SVMs work well in higher dimensional settings (many features) or with complex data
      - SVMs are flexible enough to deal with non-linearly separable or even very complex data due to the use of an error term or possible choice of different kernels
      - On a similar note, SVMs are robust to noise, because of the error term and the idea of maximizing margins
      
   - **Weaknesses:**
      - SVMs can be expensive to train (see also our example below) - especially for higher dimensional problems
      - May require rescaled data due to the geometric nature of SVM (hyperplanes) - for example if we have a mixture of continuous and categorical input data (otherwise there might also be numerical stability issues)
   - **Why is this a good model?:**
      - SVM could work well in this case because we do not have a large amount of training data (also in comparison with the number of features
      - Also, performance is not so much of an issue here because we are not dealing with a real-time application or too many testing data points


* **k-Nearest Neighbor**
   - **Real world application**
   An interesting application of KNN that I would is fault detection or isolation in semiconductors. Semi-conductor functionality relies on the produced material to be an extremely regular lattice of atoms. Some recent studies (http://fumblog.um.ac.ir/gallery/170/Fault%20Detection%20Using%20the%20k-Nearest%20Neighbor%20Rule%20for%20Semiconductor%20Manufacturing%20Processes.pdf) suggest using KNN to determine whether a new batch of semiconductors is faulty or not. Features used here include some aspects of the production process as well as certain measurements of the batch. By understanding the processes and results of semiconductor production better one can devise ways to make production more reliable and reduce scrap etc.
   - Another applications lies in text mining (e.g. spam detection). Generelly I would consider it a good benchmark model when looking for a fitting algorithm for a given application.

   - **Strengths:**
      - KNN is very simple to interpret or explain
      - Due to the simplicity of the idea KNN is quite flexible (multi-purpose)
      - Training KNN is not very expensive
      - KNN can be used for classification but also regression
   - **Weaknesses:**
      - Running on new/testing data can be more expensive than with other algorithms
      - KNN has difficulties in higher dimensions due to the curse of dimensionality: a lot of training points are required to have sufficient coverage of space to find sufficiently close training points for testing data
   - **Why is this a good model?:**
      - I would expect KNN to do well here because the feature space is not too high dimensional or too diverse
      - Also, we do not need to evaluate the model in real time or for too many training points (only several hundred students)


*Further sources for the advantages and disadvantages analysis:*
- http://scikit-learn.org/stable/modules/tree.html
- http://scikit-learn.org/stable/modules/svm.html
- http://scikit-learn.org/stable/modules/neighbors.html
- https://www.quora.com/What-are-the-advantages-of-different-classification-algorithm
- http://www.lauradhamilton.com/machine-learning-algorithm-cheat-sheet

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [240]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.\n".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [241]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


# TODO: Initialize the three models
clf_DTC = DecisionTreeClassifier(random_state=15, max_depth=3)
clf_SVM = SVC(random_state=15)
clf_KNN = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_all,y_all, train_size = 100, random_state=15)
X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_all,y_all, train_size = 200, random_state=15)
X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_all,y_all, train_size = 300, random_state=15)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_DTC, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_DTC, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_DTC, X_train_300, y_train_300, X_test_300, y_test_300)

train_predict(clf_SVM, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_SVM, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_SVM, X_train_300, y_train_300, X_test_300, y_test_300)

train_predict(clf_KNN, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_KNN, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_KNN, X_train_300, y_train_300, X_test_300, y_test_300)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8794.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7202.

Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0007 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8581.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.8136.

Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8392.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7971.

Training a SVC using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8452.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7951.

Training a SVC using a training set size of 200

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Tree Classifier (max_depth=3)**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0007       |         0.0002         |    0.8794        |    0.7202       |
| 200               |            0.0006       |         0.0002         |    0.8581        |    0.8136       |
| 300               |            0.0012       |         0.0002         |    0.8392        |    0.7971       |

** Classifer 2 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0010       |        0.0018           |   0.8452         |       0.7951     |
| 200               |     0.0031             |     0.0021             |     0.8707       |    0.7974      |
| 300               |      0.0058              |   0.0016                |   0.8712        |     0.8027    |

** Classifer 3 - k-Nearest Neighbor**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0004        |          0.0026        |     0.8516       |    0.7696      |
| 200               |      0.0007               |   0.0023               |     0.8667       |   0.7829       |
| 300               |       0.0014           |      0.0021            |     0.8630       |     0.8321    |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
In this situation I would recommend using the decision tree model. In my experiments with three quite different models (Decision Tree, K-Nearest-Neighbors and Support Vector machines) there was no clear winner on accuracy to be determined. All models show accuracy scores of around 80%. However, the decision tree model is clearly the fastest out of the three considering both training and prediction time. In terms of accuracy the model performs in line with the other models (note that we had already restricted the `'max_depth`' to prevent overfitting).

This choice of model is supported by the fact that it is used in similar situations like loan approval/credit risk analysis. Note that because of the comparable simplicity of the model changing data or retraining the algorithm (e.g. year by year) is easy.

Generally, when looking for the optimal algorithm to use one has to balance accuracy and performance/speed. In cases where accuracy is paramount but prediction time is less important, one would choose more complex, computationally intensive algorithms and tailor them in detail (e.g. by using extensive grid search or the like). However, when different algorithms do not show much deviation in terms of accuracy, then choosing the fastest one is certainly sensible.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
The decision tree algorithm is very intuitive and can be outlined with a simple example. Suppose we would like to come up with a model to decide whether or not to buy a pair of pants. The first question might be what the price is in a certain interval (say 60 - 100 \$
since we are looking for good quality but only have a maximum budget of 100 \$) and the second question whether they fit or not. So in a shop we could potentially find a pair of pants at 70 \$, then try them on, and if they fit buy them straight away.

In more precise terms the decision tree model works as follows. We are given a training data (a group of previous students) and a list of features, or information points on them. The algorithm now looks at one information point, say father's education. No it divides the groups up such that one group (e.g. father's education class < 4) is clearly more likely to fail than the other (e.g. father's education > level 3). In the next step it looks, for example, at the group of students with father's education < 3 and finds another criterion to help separate the better students from the worse, say access to internet. It may turn out that students with access to internet are more successful since they use the internet as an information source.

This process goes on until a sufficiently detailed segmentation of students along the different information points is achieved. Now to predict whether a student is likely to fail, the algorithm just needs to follow the same logic as in training to determine which group a student belongs to.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [242]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer, f1_score
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {'max_depth': range(1,10), 'min_samples_leaf': range(2,10), 'min_samples_split': range(1,10)}

#cv_sets = ShuffleSplit(X_all.shape[0], n_iter = 10, train_size = 300, random_state = 0)

# TODO: Initialize the classifier
clf = DecisionTreeClassifier(random_state=15)
# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print "The optimized parameter values are as follows:"
print grid_obj.best_params_, "\n"
# Report the final F1 score for training and testing after parameter tuning

print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

The optimized parameter values are as follows:
{'min_samples_split': 1, 'max_depth': 3, 'min_samples_leaf': 7} 

Made predictions in 0.0006 seconds.
Tuned model has a training F1 score of 0.8357.
Made predictions in 0.0001 seconds.
Tuned model has a testing F1 score of 0.8029.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final score of the optimized model on the training data is 0.8357 and on the testing data: 0.8029, compared with 0.8392, respectively, 0.7971 for the previously used decision tree model (on 300 training points). So, all in all the parameter optimization with grid search did not yield a significant improvement in accuracy.

*Note 1:* Here we did not use k-fold cross validation in the grid search algorithm. So there is some uncertainty in the individual scores and therefore possibly also in the optimized parameter set.

*Note 2:* Also for the other models (SVM and KNN) optmizing the parameters did not yield much improvement in terms of accuracy. This leads me to the conclusion that around 80 % accuracy is what "off-the-shelf" models can achieve here

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.